# 1. Large Datasets and Where to Find Them

In [18]:
from transformers import pipeline
from datasets import load_dataset

from tqdm import tqdm

In [2]:
generation_gpt = pipeline("text-generation", model="openai-gpt")

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457M [00:00<?, ?B/s]

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/448k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

In [3]:
from transformers import AutoTokenizer

In [4]:
python_code = r"""def say_hello():
      print("Hello, World!")
# Print it
say_hello()
"""

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [6]:
tokenizer(python_code).tokens()

['def',
 'Ġsay',
 '_',
 'hello',
 '():',
 'Ċ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġ',
 'Ġprint',
 '("',
 'Hello',
 ',',
 'ĠWorld',
 '!"',
 ')',
 'Ċ',
 '#',
 'ĠPrint',
 'Ġit',
 'Ċ',
 'say',
 '_',
 'hello',
 '()',
 'Ċ']

# 2. Building a Tokenizer

In [1]:
a, e = u"a", u"€"

In [3]:
byte = ord(a.encode("utf-8"))

In [4]:
byte

97

In [5]:
byte = [ord(chr(i)) for i in e.encode("utf-8")]

In [6]:
byte

[226, 130, 172]

In [7]:
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

In [10]:
byte_to_unicode_map = bytes_to_unicode()

In [12]:
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())

In [7]:
#unicode_to_byte_mapa

In [14]:
base_vocab = list(unicode_to_byte_map.keys())

In [21]:
#base_vocab

In [30]:
tokens = sorted(tokenizer.vocab.items(), key=lambda x: x[1])

In [31]:
[{tokenizer.convert_tokens_to_string(t)} for t, _ in tokens[:10]]

[{'!'}, {'"'}, {'#'}, {'$'}, {'%'}, {'&'}, {"'"}, {'('}, {')'}, {'*'}]

In [33]:
#tokens

### new

In [6]:
from transformers.models.gpt2.tokenization_gpt2 import bytes_to_unicode

byte_to_unicode_map = bytes_to_unicode()
unicode_to_byte_map = dict((v, k) for k, v in byte_to_unicode_map.items())
base_vocab = list(unicode_to_byte_map.keys())

In [7]:
from tqdm.auto import tqdm
from datasets import load_dataset

In [8]:
length = 100000

In [11]:
dataset_name = "transformersbook/codeparrot-train"

In [19]:
dataset = load_dataset(dataset_name, split="train", streaming=True)

NameError: name 'dataset_name' is not defined

In [ ]:
iter_dataset = iter(dataset)

In [ ]:
def batch_iterator(batch_size=10):
    for _ in tqdm(range(0, length, batch_size)):
        yield [next(iter_dataset)['content'] for _ in range(batch_size)]

In [ ]:
batch_iterator()

In [ ]:
new_tokenizer = tokenizer.train_new_from_iterator(
    batch_iterator(),
    vocab_size=12500,
    initial_alphabet=base_vocab
)

### Training a Model from Scratch

In [21]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

In [22]:
model_ckpt = "gpt2-xl"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [24]:
config = AutoConfig.from_pretrained("gpt2-xl", vocab_size = len(tokenizer))

In [25]:
config

GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 50257
}

In [26]:
model = AutoModelForCausalLM.from_config(config)

In [27]:
examples, total_characters, total_tokens = 500, 0, 0

In [28]:
dataset = load_dataset(
    "transformersbook/codeparrot-train",
    split="train",
    streaming=True
)

Using custom data configuration transformersbook--codeparrot-train-39fd2cee2b2cb397


In [29]:
for _, example in tqdm(zip(range(examples), iter(dataset)), total=examples):
    total_characters += len(example["content"])
    total_tokens += len(tokenizer(example["content"]).tokens())

100%|██████████| 500/500 [00:06<00:00, 78.63it/s] 


In [30]:
total_characters

5568371

In [31]:
total_tokens

2527740

In [32]:
import torch
from torch.utils.data import IterableDataset

In [ ]:
# class ConstantLengthDataset(IterableDataset):
#     def __init__(
#         self, tokenizer, dataset, seq_length=1024,
#         num_of_sequences=1024, chars_per_token=3.6
#     ):
#         self.tokenizer = tokenizerenizer
#         self.concat_token_id = tokenizer.eos_token_id
#         self.dataset = dataset
#         self.seq_length = seq_length
#         self.input_characters = seq_length * char_per_token * num_of_sequences
    
#     def __iter__(self):
#         iterator = iter(self.dataset)
#         more_examples = True
        
#         while more_examples:
#             buffer, buffer_len = [], 0
            
#             while True:
#                 if buffer_len >= self.input_characters:
#                     m = f"Buffer full: {}"

In [33]:
class ConstantLengthDataset(IterableDataset):
    
    def __init__(self, tokenizer, dataset, seq_length=1024,
                 num_of_sequences=1024, chars_per_token=3.6):
        self.tokenizer = tokenizer
        self.concat_token_id = tokenizer.eos_token_id
        self.dataset = dataset
        self.seq_length = seq_length
        self.input_characters = seq_length * chars_per_token * num_of_sequences
    
    def __iter__(self):
        iterator = iter(self.dataset)
        more_examples = True
        while more_examples:
            buffer, buffer_len = [], 0
            while True:
                if buffer_len >= self.input_characters:
                    m=f"Buffer full: {buffer_len}>={self.input_characters:.0f}"
                    print(m)
                    break
                try:
                    m=f"Fill buffer: {buffer_len}<{self.input_characters:.0f}"
                    print(m)
                    buffer.append(next(iterator)["content"])
                    buffer_len += len(buffer[-1])
                except StopIteration:
                    iterator = iter(self.dataset)

            all_token_ids = []
            tokenized_inputs = self.tokenizer(buffer, truncation=False)
            for tokenized_input in tokenized_inputs['input_ids']:
                all_token_ids.extend(tokenized_input + [self.concat_token_id])
            
            for i in range(0, len(all_token_ids), self.seq_length):
                input_ids = all_token_ids[i : i + self.seq_length]
                if len(input_ids) == self.seq_length:
                    yield torch.tensor(input_ids)

In [34]:
shuffled_dataset = dataset.shuffle(buffer_size=100)

In [ ]:
constant_length_dataset = ConstantLengthDataset(
    tokenizer, shuffled_dataset,
    num_of_sequences=10
)

In [9]:
constant_length_dataset.__dict__

NameError: name 'constant_length_dataset' is not defined

In [ ]:
dataset_iterator = iter(constant_length_dataset)

In [ ]:
lengths = [len(b) for _, b in zip(range(5), dataset_iterator)]

In [ ]:
lengths

In [ ]:
#list(dataset_iterator)

In [38]:
from argparse import Namespace

# Commented parameters correspond to the small model
config = {"train_batch_size": 2, # 12
          "valid_batch_size": 2, # 12
          "weight_decay": 0.1,
          "shuffle_buffer": 1000,
          "learning_rate": 2e-4, # 5e-4
          "lr_scheduler_type": "cosine",
          "num_warmup_steps": 750, # 2000
          "gradient_accumulation_steps": 16, # 1
          "max_train_steps": 50000, # 150000
          "max_eval_steps": -1,
          "seq_length": 1024,
          "seed": 1,
          "save_checkpoint_steps": 50000} # 15000

args = Namespace(**config)

In [40]:
from torch.utils.data.dataloader import DataLoader

In [41]:
def create_dataloaders(dataset_name):
    train_data = load_dataset(
        dataset_name+'-train',
        split="train", streaming=True
    )
    valid_data = load_dataset(
        dataset_name+'-valid',
        split="validation", streaming=True
    )
    
    train_data = train_data.shuffle(
        buffer_size=args.shuffle_buffer,
        seed=args.seed
    )
    
    train_dataset = ConstantLengthDataset(
        tokenizer, train_data, seq_length=args.seq_length
    )
    valid_dataset = ConstantLengthDataset(
        tokenizer, valid_data, seq_length=args.seq_length
    )

### Results and Analysis

In [42]:
from transformers import pipeline, set_seed

In [43]:
model_ckpt = "transformersbook/codeparrot-small"

In [44]:
generation = pipeline("text-generation", model=model_ckpt)

Downloading:   0%|          | 0.00/865 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/259 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/270k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/821k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [45]:
import re
from transformers import set_seed

In [46]:
def first_block(string):
    return re.split('\nclass|\ndef|\n#|\n@|\nprint|\nif', string)[0].rstrip()

In [55]:
def complete_code(pipe, promopt, max_length=64, num_completions=4):
    gen_kwargs = {
        "temperature": 0.4,
        "top_p": 0.95,
        "top_k": 0,
        "num_beams": 1,
        "do_sample": True
    }
    
    code_gens = generation(
        prompt, num_return_sequences=num_completions,
        max_length=max_length, **gen_kwargs
    )
    
    code_strings = []
    for code_gen in code_gens:
        generated_code = first_block(code_gen["generated_text"][len(prompt):])
        code_strings.append(generated_code)
    print(('\n'+'='*80 + '\n').join(code_strings))

In [56]:
prompt = '''def area_of_rectangle(a: float, b: float):
      """Return the area of the rectangle."""'''

In [57]:
complete_code(generation, prompt)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



      return a / b

  def area_of_rectangle_area(a: float, b: float):
      """Return the area of the rectangle area."""
      return a / b

  def

      return a / b

  def area_of_rectangle_from_rectangle(a: float, b: float):
      """Return the area of the rectangle from a rectangle."""
      return a

      return self.area_of_rectangle(a, b)

  def area_of_rectangle(self, a: float, b: float):
      """Return the area of the rectangle

      return a * b

  def area_of_rectangle_in_polygon(a: float, b: float):
      """Return the area of the rectangle in polygon."""
      return a *
